In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
list_news = soup.find('ul', class_='item_list')
last_news = list_news.find_all('li')[0]
link_news = last_news.find('a')['href']
print(link_news)

/news/8508/nasa-invites-students-to-name-mars-2020-rover/


In [3]:
url = 'https://mars.nasa.gov' + link_news
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
title = soup.find('h1', class_='article_title')
title = title.text
title = title[1:]
title = title[:-1]
title

'NASA Invites Students to Name Mars 2020 Rover'

In [4]:
par_list = soup.find('div', class_='wysiwyg_content')
# Get the first paragraph
par = par_list.find_all('p')[0]
para = par.text
para

'Red rover, red rover, send a name for Mars 2020 right over! NASA is recruiting help from students nationwide to find a name for its next Mars rover mission.'

In [5]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
carousel_item = soup.find('article', class_="carousel_item")
footer = carousel_item.find('footer')
fig_id = footer.find('a')['data-link']
fig_id = fig_id.split("=")[1]
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/' + fig_id + '_ip.jpg'
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17936_ip.jpg'

In [7]:
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [8]:
stream = soup.find('ol', class_='stream-items js-navigable-stream')
stream2 = stream.find_all('li')[0]
stream3 = stream2.find('div', class_='js-tweet-text-container')
mars_weather = stream3.find('p').text
print (mars_weather)

We won’t be hearing from @MarsCuriosity or @NASAInSight for the next 2 weeks during Mars solar conjunction. Read more about why Mars missions go silent every 2 years: https://www.wral.com/mars-spacecraft-go-quiet-during-solar-conjunction/18595551/ …pic.twitter.com/fWruE2v151


In [10]:
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)
df = tables[1]
df.rename(columns={0:'Info',1:'Value'}, inplace=True)
df

,Info,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [11]:
html_table = df.to_html()
html_table = html_table.replace('\n', "")
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Info</th>      <th>Value</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td

In [12]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
urls =[]
list = soup.find('div', class_='collapsible results')
lists = list.find_all('div', class_="item")

for item in lists:
    desc = item.find('div', class_='description')
    urls.append('https://astrogeology.usgs.gov' +  desc.find('a')['href'])
urls

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [13]:
hemi = []
for url in urls:
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    html = soup.find('div', class_='downloads')
    img = 'https://astrogeology.usgs.gov' + html.find('img')['src']
    title = soup.title.text
    title = title.split("|")[0]
    title = title.replace("Enhanced","")
    hemi.append({"title":title, "img": img})  
print(hemi)

[{'title': 'Cerberus Hemisphere  ', 'img': 'https://astrogeology.usgs.gov/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png'}, {'title': 'Schiaparelli Hemisphere  ', 'img': 'https://astrogeology.usgs.gov/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png'}, {'title': 'Syrtis Major Hemisphere  ', 'img': 'https://astrogeology.usgs.gov/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png'}, {'title': 'Valles Marineris Hemisphere  ', 'img': 'https://astrogeology.usgs.gov/cache/images/04085d99ec3713883a9a57f42be9c725_valles_marineris_enhanced.tif_thumb.png'}]
